## <CENTER> Notebook test
______________________________________________

### This is an example notebook using ROOT libraries from python. You can play around here and experiment, and you can also create several separate notebooks for specific studies.

### Best of luck!
_____________________________________________

In [ ]:
# to use standard python modules, you import them
import math

# in order to use ROOT, you import ROOT
import ROOT

### Make a histogram and fill it with random numbers

In [ ]:
# let's create a one-dimensional histogram with 10 bins in x, covering -5 to 5.
h = ROOT.TH1F("randomnumbers", "My random numbers", 10, -5, 5)

In [ ]:
# now let's make a pseudo-random number generator and have it generate numbers following a normal distribution
prng = ROOT.TRandom3(123) # 123 is the seed which sets the initial state of the prng
print("test")
for i in range(10):
    print("Number %d: %f" % (i, prng.Gaus()))


In [ ]:
# now fill the histogram we just made with random numbers: 
for i in range(100000):
    h.Fill(prng.Gaus())

In [ ]:
# to draw the histogram we need to define a canvas etc.
c1 = ROOT.TCanvas("c1","c1",900,600)
h.Draw()
c1.Draw()

### Now let's try some other distributions

In [ ]:
h_pois = ROOT.TH1F("poisson", "Poissonian numbers", 50, 0, 50)
h_land = ROOT.TH1F("landau", "Landau numbers", 100, 0, 20)

In [ ]:
pois_lambda = 15
for i in range(100000):
    h_pois.Fill(prng.Poisson(pois_lambda))

In [ ]:
landau_mpv, landau_sigma = 5,1
for i in range(100000):
    h_land.Fill(prng.Landau(landau_mpv, landau_sigma))

In [ ]:
# to draw the histogram we need to define a canvas etc.
c2 = ROOT.TCanvas("c2","c2",900,600)
h_pois.Draw()
c2.Draw()

In [ ]:
c3 = ROOT.TCanvas("c3","c3",900,600)
h_land.Draw()
c3.Draw()

### We can also combine histograms!

In [ ]:
# let's define a total histogram, and two histograms representing two different contributions, 
# each described by an exponential distribution
h_exp = ROOT.TH1F("exponential", "Exponential combined distribution", 100, 0, 250)
h_exp_low = ROOT.TH1F("exp_low", "Exponential low distribution", 100, 0, 250)
h_exp_high = ROOT.TH1F("exp_high", "Exponential high distribution", 100, 0, 250)

# the two exponentials have different values for the parameter that describes the slope
exp_slope_high = 50
exp_slope_low  = 14

In [ ]:
# the probabilities of the two processes that the contributions correspond to can be different, 
# let's say the probability of the process corresponding to the high contribution is 0.346%
prob_of_high = 0.00346
print(prob_of_high)

In [ ]:
for i in range(1000000): # generate 1M random numbers for the combination of the two contributions
    if prng.Rndm() > prob_of_high: # the random number generated this way is uniform in [0,1]
        low_exp = prng.Exp(exp_slope_low)
        h_exp.Fill(prng.Exp(exp_slope_low))
        h_exp_low.Fill(low_exp)
    else:
        high_exp = prng.Exp(exp_slope_high)
        h_exp.Fill(high_exp)
        h_exp_high.Fill(high_exp)

In [ ]:
c4 = ROOT.TCanvas("c4","c4",900,600)
c4.SetLogy() # let's use logarithmic y-axis to see the tail in detail
h_exp_low.Draw()
h_exp_high.Draw("SAME")
h_exp_low.SetLineColor(2)
c4.BuildLegend() # build a legend explaining the two histograms
ROOT.gStyle.SetOptStat(0) # turn off the box with statistics, i.e. number of entries etc
c4.Draw()

In [ ]:
c5 = ROOT.TCanvas("c5","c5",900,600)
h_exp.Draw() # this is the sum, we filled it for both processes
c5.SetLogy()
c5.Draw()

## Defining functions

In [ ]:
# we can define a function describing the Poisson distribution
# this defines a Poisson with mu=50 over the interval [0, 500]
fPoisson = ROOT.TF1("myPoisson", "TMath::Poisson(x, [mu])", 0, 500)
fPoisson.SetParameter("mu", 50)
fPoisson.SetNpx(1000) # use higher number of points than default (100) when drawing
c6 = ROOT.TCanvas("c6", "c6", 800, 600)
fPoisson.Draw()
c6.Draw()

In [ ]:
# print out some info about the function
fPoisson.Print()

In [ ]:
# check the parameters
for p in range(0, fPoisson.GetNpar()):
    print("%d: %s = %f" % (p, fPoisson.GetParName(p), fPoisson.GetParameter(p)))

In [ ]:
# let's also define a gaussian
fGauss = ROOT.TF1("myGauss", "gaus", 0, 500)
fGauss.Print()

# check the parameter values - we haven't set them, so these are the default values
for p in range(0, fGauss.GetNpar()):
    print("%d: %s = %f" % (p, fGauss.GetParName(p), fGauss.GetParameter(p)))

In [ ]:
# set the parameter values
fGauss.SetParameter("Constant", 0.05) # to easily see it next to the Poisson
fGauss.SetParameter("Mean", 14)
fGauss.SetParameter("Sigma", 2)
fGauss.SetNpx(1000) # use higher number of points than default (100) when drawing
c6.cd()
fGauss.Draw("SAME")
c6.Draw()


In [ ]:
# let's make the y axis logarithmic to more easily see both large and small values
c6.SetLogy(1)
c6.Draw()

# Now to the task of generating the distribution of tracks per BC
What do we know?
* The number of interactions per BC follows a Poisson distribution, like `fPoisson` above
* The number of tracks per interaction follows the sum of two exponential terms

The trick is now to combine these to generate toy BCs, by drawing a random number of interactions, and for each interaction in the BC generate a random number of tracks!